<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=f368c44200bc84901445945b36496e4b602777871bec92df750d559fa0369fe9
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-1000)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Mounted at /content/drive


In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

M    71
L    63
H    37
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0    104
1     67
Name: StarStock, dtype: int64

In [7]:
# v40/v40n stocks
mypf['Criteria'].value_counts()

V40N    39
V40     38
2T3Y     5
SS       4
TBD      1
Name: Criteria, dtype: int64

In [8]:
# latest quarter
mypf['LatestQtr'].value_counts()

1    116
0     55
Name: LatestQtr, dtype: int64

In [9]:
# star + v40/v40n stocks
mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]['StarStock'].value_counts()

0    46
1    31
Name: StarStock, dtype: int64

In [10]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    148
1     23
Name: InPortfolio, dtype: int64

In [11]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [12]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [13]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    #stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0] 
    tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]  
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]       
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0] 
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [14]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-04-20 00:00:00+05:30,17624.0,^NSEI,0.16,0.60,0.03,18812.0,10806.0,14.84,7.0
2023-04-27 00:00:00+05:30,17915.0,^NSEI,1.53,0.57,-0.03,18812.0,10806.0,11.20,5.0
2023-05-05 00:00:00+05:30,18069.0,^NSEI,2.11,0.56,0.12,18812.0,10806.0,9.28,4.0
2023-05-12 00:00:00+05:30,18315.0,^NSEI,3.18,0.59,0.31,18812.0,10806.0,6.21,3.0
2023-05-19 00:00:00+05:30,18203.0,^NSEI,2.32,0.54,0.48,18812.0,10806.0,7.61,3.0


In [15]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [16]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [17]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 23


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1066.0,-37.57,-3.05,0.00,100.00,305.0,1354.78,45.2,21.3,M,53.0,1,0,2T3Y,2T3Y
VALIANTORG.NS,485.0,-15.64,-1.39,1.52,92.86,282.0,481.73,24.0,15.5,M,35.0,0,0,2T3Y,2T3Y
VAIBHAVGBL.NS,296.0,-7.71,0.00,-0.33,96.40,226.0,320.64,35.4,46.6,M,-32.0,1,0,2T3Y,2T3Y
LUXIND.NS,1446.0,-8.70,-1.20,1.21,90.72,214.0,1499.87,21.4,24.1,M,-13.0,0,0,V40N,2T3Y
INDIGOPNTS.NS,1404.0,6.99,-0.30,3.19,80.26,122.0,1215.89,104.9,56.7,M,46.0,0,1,V40N,2T3Y
JUBLFOOD.NS,469.0,-10.38,-0.96,0.89,88.75,93.0,468.21,90.5,87.6,H,3.0,1,1,SS,ATH
IRCTC.NS,610.0,-7.80,0.30,0.17,61.08,91.0,629.08,77.4,51.8,H,33.0,0,1,SS,ATH
WIPRO.NS,386.0,-2.23,-0.25,0.00,73.15,85.0,382.90,20.9,18.7,L,11.0,1,0,2T3Y,2T3Y
HDFCAMC.NS,1805.0,-8.26,-0.20,0.11,91.05,79.0,1856.05,38.5,27.1,M,30.0,1,0,V40,ATH


In [18]:
# top 7 from portfolio for SIP
tmp_df.sort_values(by = 'Upside%ATH', ascending=False).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,1066.0,-37.57,-3.05,0.00,100.00,305.0,1354.78,45.2,21.3,M,53.0,1,0,2T3Y,2T3Y
VALIANTORG.NS,485.0,-15.64,-1.39,1.52,92.86,282.0,481.73,24.0,15.5,M,35.0,0,0,2T3Y,2T3Y
VAIBHAVGBL.NS,296.0,-7.71,0.00,-0.33,96.40,226.0,320.64,35.4,46.6,M,-32.0,1,0,2T3Y,2T3Y
LUXIND.NS,1446.0,-8.70,-1.20,1.21,90.72,214.0,1499.87,21.4,24.1,M,-13.0,0,0,V40N,2T3Y
INDIGOPNTS.NS,1404.0,6.99,-0.30,3.19,80.26,122.0,1215.89,104.9,56.7,M,46.0,0,1,V40N,2T3Y
JUBLFOOD.NS,469.0,-10.38,-0.96,0.89,88.75,93.0,468.21,90.5,87.6,H,3.0,1,1,SS,ATH
IRCTC.NS,610.0,-7.80,0.30,0.17,61.08,91.0,629.08,77.4,51.8,H,33.0,0,1,SS,ATH


In [19]:
# top 7 from portfolio to book profits
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
EICHERMOT.NS,3571.0,7.40,0.75,1.47,15.11,8.0,3336.21,44.7,33.5,H,25.0,1,1,V40N,ATH
TATACONSUM.NS,765.0,1.12,0.00,0.97,24.51,13.0,758.64,68.8,66.0,H,4.0,1,1,SS,ATH
PIDILITIND.NS,2567.0,0.89,0.47,1.08,22.13,13.0,2442.09,94.6,102.0,H,-8.0,1,1,V40,ATH
RELIANCE.NS,2442.0,-1.26,0.08,0.42,37.65,15.0,2464.49,29.1,24.8,H,15.0,1,1,V40,ATH
ASIANPAINT.NS,3084.0,0.37,0.13,1.07,28.44,16.0,2954.17,87.9,71.4,H,19.0,1,1,V40,ATH
BAJFINANCE.NS,6784.0,2.65,0.47,0.99,22.85,16.0,6365.23,52.6,35.7,H,32.0,1,1,V40,ATH
PAGEIND.NS,41675.0,-5.79,-0.28,1.11,33.30,29.0,39840.50,97.7,68.0,H,30.0,0,1,V40,ATH


In [20]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [21]:
# star + v40/v40n stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
TEAMLEASE.NS,2263.0,-14.98,-2.63,-1.16,5281.0,2028.0,92.78,133.0,NaN,NaN,M,1,1,V40N,NaN
BERGEPAINT.NS,625.0,3.35,0.33,0.84,858.0,533.0,71.69,37.0,1.0,0.0,M,1,1,V40,NaN
BAJAJFINSV.NS,1415.0,-6.47,0.53,0.52,1907.0,542.0,36.04,35.0,0.0,0.0,M,1,1,V40,NaN
SUNTV.NS,426.0,-9.38,0.21,0.23,577.0,388.0,79.89,35.0,NaN,NaN,M,1,1,V40N,NaN
UJJIVANSFB.NS,33.0,25.58,3.85,0.00,42.0,13.0,31.03,27.0,0.0,1.0,M,1,1,V40N,NaN
DABUR.NS,519.0,-5.96,-0.18,0.00,645.0,464.0,69.61,24.0,1.0,0.0,H,1,1,V40,NaN
OFSS.NS,3550.0,19.33,0.87,1.82,4392.0,2442.0,43.18,24.0,1.0,0.0,M,1,1,V40N,NaN
MCDOWELL-N.NS,817.0,-0.60,-0.12,0.78,1004.0,496.0,36.81,23.0,0.0,1.0,L,1,1,V40N,NaN
TCS.NS,3223.0,-0.00,0.37,-0.37,3886.0,2132.0,37.80,21.0,1.0,0.0,H,1,1,V40,NaN


In [22]:
# top 5 from star + v40/v40n for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
TEAMLEASE.NS,2263.0,-14.98,-2.63,-1.16,5281.0,2028.0,92.78,133.0,NaN,NaN,M,1,1,V40N,NaN
SUNTV.NS,426.0,-9.38,0.21,0.23,577.0,388.0,79.89,35.0,NaN,NaN,M,1,1,V40N,NaN
BAJAJFINSV.NS,1415.0,-6.47,0.53,0.52,1907.0,542.0,36.04,35.0,0.0,0.0,M,1,1,V40,NaN
DABUR.NS,519.0,-5.96,-0.18,0.00,645.0,464.0,69.61,24.0,1.0,0.0,H,1,1,V40,NaN
MCDOWELL-N.NS,817.0,-0.60,-0.12,0.78,1004.0,496.0,36.81,23.0,0.0,1.0,L,1,1,V40N,NaN


In [23]:
# star + v40/v40n stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
HDFCBANK.NS,1647.0,6.05,0.97,0.43,1708.0,1003.0,8.65,4.0,1.0,0.0,H,1,1,V40,NaN
HINDUNILVR.NS,2641.0,2.86,0.08,0.71,2728.0,1897.0,10.47,3.0,1.0,1.0,H,1,1,V40,NaN
ITC.NS,420.0,19.62,1.71,1.00,431.0,146.0,3.86,3.0,1.0,1.0,H,1,1,V40,NaN
TITAN.NS,2702.0,5.86,1.02,1.44,2790.0,1090.0,5.18,3.0,1.0,1.0,H,1,1,V40,NaN
BAJAJ-AUTO.NS,4483.0,16.48,0.73,1.93,4565.0,2649.0,4.28,2.0,NaN,NaN,L,1,1,V40,NaN
EQUITASBNK.NS,81.0,42.46,3.51,1.41,83.0,33.0,4.00,2.0,1.0,1.0,M,1,1,V40N,NaN
CERA.NS,7497.0,32.18,2.50,1.51,7497.0,2229.0,0.00,0.0,1.0,1.0,H,1,1,V40N,NaN
ICICIBANK.NS,954.0,7.95,0.90,1.12,954.0,333.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN
POLYCAB.NS,3441.0,24.22,2.06,1.13,3441.0,793.0,0.00,0.0,1.0,0.0,M,1,1,V40N,NaN


In [24]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ADANIGREEN.NS,897.0,-45.37,-3.71,4.37,2970.0,414.0,81.10,231.0,-1.0,-1.0,L,1,1,NaN,NaN
ADANIENT.NS,1956.0,-32.39,-0.93,2.70,4165.0,248.0,56.40,113.0,1.0,0.0,M,1,1,NaN,NaN
BANDHANBNK.NS,238.0,-2.88,-0.82,0.46,422.0,187.0,78.30,77.0,1.0,1.0,L,1,1,NaN,NaN
LTIM.NS,4803.0,4.73,0.48,-0.04,7439.0,2267.0,50.97,55.0,1.0,0.0,M,1,1,NaN,NaN
TATAPOWER.NS,206.0,-4.26,-0.47,0.00,287.0,49.0,34.03,39.0,1.0,0.0,M,1,1,NaN,NaN
GODREJIND.NS,470.0,6.56,0.23,1.15,644.0,354.0,60.00,37.0,0.0,1.0,L,1,1,NaN,NaN
CIPLA.NS,916.0,-10.78,-0.19,0.00,1173.0,701.0,54.45,28.0,1.0,0.0,M,1,1,NaN,NaN
SBICARD.NS,892.0,8.86,0.00,1.82,1133.0,673.0,52.39,27.0,1.0,0.0,M,1,1,NaN,NaN
UPL.NS,668.0,-8.49,-0.14,-0.56,832.0,391.0,37.19,25.0,NaN,NaN,L,1,1,NaN,NaN


In [25]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 11


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
AUBANK.NS,754.0,19.27,1.11,2.04,754.0,317.0,0.00,0.0,1.0,0.0,M,1,1,NaN,NaN
CERA.NS,7497.0,32.18,2.50,1.51,7497.0,2229.0,0.00,0.0,1.0,1.0,H,1,1,V40N,NaN
ICICIBANK.NS,954.0,7.95,0.90,1.12,954.0,333.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN
POLYCAB.NS,3441.0,24.22,2.06,1.13,3441.0,793.0,0.00,0.0,1.0,0.0,M,1,1,V40N,NaN
INDUSINDBK.NS,1248.0,9.28,1.40,1.26,1264.0,484.0,2.05,1.0,1.0,1.0,H,1,1,NaN,NaN
EQUITASBNK.NS,81.0,42.46,3.51,1.41,83.0,33.0,4.00,2.0,1.0,1.0,M,1,1,V40N,NaN
BRITANNIA.NS,4500.0,9.86,1.12,0.57,4635.0,2945.0,7.99,3.0,1.0,1.0,H,1,1,NaN,NaN
HINDUNILVR.NS,2641.0,2.86,0.08,0.71,2728.0,1897.0,10.47,3.0,1.0,1.0,H,1,1,V40,NaN
ITC.NS,420.0,19.62,1.71,1.00,431.0,146.0,3.86,3.0,1.0,1.0,H,1,1,V40,NaN


In [26]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ACC.NS,1729.0,-20.47,-0.92,0.06,2748.0,1259.0,68.44,59.0,0.0,0.0,L,0,0,NaN,NaN
PGHH.NS,13591.0,-2.48,-0.03,0.01,15778.0,9190.0,33.20,16.0,0.0,0.0,L,0,0,V40,NaN
PVRINOX.NS,1373.0,-4.55,-3.13,-1.18,1483.0,1367.0,94.83,8.0,1.0,-1.0,L,0,0,NaN,NaN
SIEMENS.NS,3723.0,22.94,1.82,1.78,3891.0,1143.0,6.11,5.0,1.0,1.0,H,0,0,NaN,NaN
NESTLEIND.NS,21690.0,10.99,0.88,1.58,22050.0,14623.0,4.85,2.0,1.0,1.0,H,0,0,V40,NaN


In [36]:
# stocks prospects 

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 4


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
BANDHANBNK.NS,238.0,-2.88,-0.82,0.46,422.0,187.0,78.30,77.0,1.0,1.0,L,1,1,NaN,NaN
GICRE.NS,174.0,19.76,2.07,2.00,223.0,108.0,42.61,28.0,NaN,NaN,M,0,0,2T3Y,2T3Y
UJJIVANSFB.NS,33.0,25.58,3.85,0.00,42.0,13.0,31.03,27.0,0.0,1.0,M,1,1,V40N,NaN
PNB.NS,49.0,7.07,2.17,0.00,60.0,26.0,32.35,22.0,NaN,NaN,L,1,1,NaN,NaN


In [27]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,145.0,-19.05,-2.23,-1.35,303.0,137.0,95.18,109.0,1.0,1.0,H,1,0,NaN,NaN
BANDHANBNK.NS,238.0,-2.88,-0.82,0.46,422.0,187.0,78.30,77.0,1.0,1.0,L,1,1,NaN,NaN


In [28]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 3


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,145.0,-19.05,-2.23,-1.35,303.0,137.0,95.18,109.0,1.0,1.0,H,1,0,NaN,NaN
BANDHANBNK.NS,238.0,-2.88,-0.82,0.46,422.0,187.0,78.30,77.0,1.0,1.0,L,1,1,NaN,NaN
HDFCLIFE.NS,558.0,2.50,0.37,1.35,757.0,461.0,67.23,36.0,1.0,1.0,H,1,0,V40,NaN


In [29]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 13


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
CERA.NS,7497.0,32.18,2.50,1.51,7497.0,2229.0,0.00,0.0,1.0,1.0,H,1,1,V40N,NaN
ICICIBANK.NS,954.0,7.95,0.90,1.12,954.0,333.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN
INDUSINDBK.NS,1248.0,9.28,1.40,1.26,1264.0,484.0,2.05,1.0,1.0,1.0,H,1,1,NaN,NaN
ITC.NS,420.0,19.62,1.71,1.00,431.0,146.0,3.86,3.0,1.0,1.0,H,1,1,V40,NaN
EQUITASBNK.NS,81.0,42.46,3.51,1.41,83.0,33.0,4.00,2.0,1.0,1.0,M,1,1,V40N,NaN
NESTLEIND.NS,21690.0,10.99,0.88,1.58,22050.0,14623.0,4.85,2.0,1.0,1.0,H,0,0,V40,NaN
TITAN.NS,2702.0,5.86,1.02,1.44,2790.0,1090.0,5.18,3.0,1.0,1.0,H,1,1,V40,NaN
SIEMENS.NS,3723.0,22.94,1.82,1.78,3891.0,1143.0,6.11,5.0,1.0,1.0,H,0,0,NaN,NaN
BANKBARODA.NS,182.0,14.40,2.52,1.72,193.0,39.0,7.14,6.0,1.0,1.0,H,1,1,NaN,NaN


In [30]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
NESTLEIND.NS,21690.0,10.99,0.88,1.58,22050.0,14623.0,4.85,2.0,1.0,1.0,H,0,0,V40,NaN
TITAN.NS,2702.0,5.86,1.02,1.44,2790.0,1090.0,5.18,3.0,1.0,1.0,H,1,1,V40,NaN
HDFCLIFE.NS,558.0,2.50,0.37,1.35,757.0,461.0,67.23,36.0,1.0,1.0,H,1,0,V40,NaN
COLPAL.NS,1609.0,5.01,0.20,1.18,1719.0,1246.0,23.26,7.0,1.0,1.0,H,1,1,V40,NaN
ICICIBANK.NS,954.0,7.95,0.90,1.12,954.0,333.0,0.00,0.0,1.0,1.0,H,1,1,V40,NaN
HINDUNILVR.NS,2641.0,2.86,0.08,0.71,2728.0,1897.0,10.47,3.0,1.0,1.0,H,1,1,V40,NaN
HAVELLS.NS,1260.0,2.24,0.16,0.66,1461.0,606.0,23.51,16.0,1.0,1.0,H,1,0,V40,NaN
MARUTI.NS,9106.0,4.00,0.31,0.54,9528.0,6217.0,12.75,5.0,1.0,1.0,H,1,0,NaN,NaN


In [31]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
BANDHANBNK.NS,238.0,-2.88,-0.82,0.46,422.0,187.0,78.30,77.0,1.0,1.0,L,1,1,NaN,NaN
INDUSTOWER.NS,145.0,-19.05,-2.23,-1.35,303.0,137.0,95.18,109.0,1.0,1.0,H,1,0,NaN,NaN


In [32]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 52


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ADANITRANS.NS,787.0,-66.74,-4.44,1.26,4105.0,235.0,85.74,422.0,1.0,-1.0,L,0,0,NaN,NaN
PEL.NS,741.0,-20.65,-5.35,-0.42,2866.0,635.0,95.25,287.0,-1.0,0.0,L,1,0,NaN,NaN
IBREALEST.NS,55.0,-23.75,-1.39,0.00,191.0,47.0,94.44,247.0,-1.0,-1.0,L,0,0,NaN,NaN
ADANIGREEN.NS,897.0,-45.37,-3.71,4.37,2970.0,414.0,81.10,231.0,-1.0,-1.0,L,1,1,NaN,NaN
JCHAC.NS,1086.0,-12.17,-1.94,-0.81,2837.0,1017.0,96.21,161.0,NaN,NaN,M,0,0,V40N,NaN
TEAMLEASE.NS,2263.0,-14.98,-2.63,-1.16,5281.0,2028.0,92.78,133.0,NaN,NaN,M,1,1,V40N,NaN
LALPATHLAB.NS,1940.0,-12.65,-0.18,0.21,4186.0,1748.0,92.12,116.0,NaN,NaN,M,1,0,V40N,NaN
ADANIENT.NS,1956.0,-32.39,-0.93,2.70,4165.0,248.0,56.40,113.0,1.0,0.0,M,1,1,NaN,NaN
BIOCON.NS,242.0,-7.38,-1.92,0.89,483.0,199.0,84.86,100.0,1.0,0.0,M,0,0,NaN,NaN


In [33]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
PVRINOX.NS,1373.0,-4.55,-3.13,-1.18,1483.0,1367.0,94.83,8.0,1.0,-1.0,L,0,0,NaN,NaN


In [34]:
# save the output
#df_prec_dev = df_prec_dev.reset_index()
#df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/Nifty100-Upside-Analysis-ATH.csv', index=False)

In [35]:
# nifty in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-04-20 00:00:00+05:30,772.0,BHARTIARTL.NS,0.56,0.65,0.00,851.0,396.0,17.36,10.0
2023-04-27 00:00:00+05:30,786.0,BHARTIARTL.NS,2.01,0.65,-0.26,851.0,396.0,14.29,8.0
2023-05-05 00:00:00+05:30,787.0,BHARTIARTL.NS,1.72,0.78,0.26,851.0,396.0,14.07,8.0
2023-05-12 00:00:00+05:30,792.0,BHARTIARTL.NS,1.92,0.77,0.26,851.0,396.0,12.97,7.0
2023-05-19 00:00:00+05:30,806.0,BHARTIARTL.NS,3.30,0.77,0.52,851.0,396.0,9.89,6.0
